In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [133]:
train = pd.read_csv('./train_basic.csv')
test = pd.read_csv('./test_basic.csv')
df = pd.concat([train, test])

In [134]:
useful = ['model', 'age', 'price', 'dereg_value', 'depreciation', 'engine_cap', 'omv']
df = df[useful].dropna()

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15434 entries, 0 to 16783
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         15434 non-null  object 
 1   age           15434 non-null  int64  
 2   price         15434 non-null  float64
 3   dereg_value   15434 non-null  float64
 4   depreciation  15434 non-null  float64
 5   engine_cap    15434 non-null  float64
 6   omv           15434 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 964.6+ KB


In [66]:
from sklearn.metrics.pairwise import cosine_similarity

def getSim(p1, p2):
    return cosine_similarity(np.array(df[p1]).reshape(-1, 1), np.array(df[p2]).reshape(-1, 1))
df['ageSim'] = getSim('age', 'age')
df['omvSim'] = getSim('omv', 'omv')
df['derSim'] = getSim('dereg_value', 'dereg_value')
df['depSim'] = getSim('depreciation', 'depreciation')

In [135]:
# train_ = df[:-5000]
train, test = df[:10000], df[10000:]

In [136]:
pred = []

for idx, row in test.iterrows():
    model = row['model']
    sameModel = train[train.model == model]
    def getSim(p1, p2):
        return np.abs(np.array(sameModel[p2]).reshape(-1, 1) - p1)
    sameModel.loc[:, 'ageSim'] = getSim(row['age'], 'age')
    sameModel.loc[:, 'omvSim'] = getSim(row['omv'], 'omv')
    sameModel.loc[:, 'derSim'] = getSim(row['dereg_value'], 'dereg_value')
    sameModel.loc[:, 'depSim'] = getSim(row['depreciation'], 'depreciation')

    if len(sameModel) > 100:
        # between the train and currently tested
        bestMatch = sameModel.sort_values(by=['ageSim', 'omvSim', 'derSim', 'depSim']).reset_index()
        if abs(row['age'] - bestMatch.loc[0, 'age']) < 12:
            price = np.average(bestMatch.loc[:10, 'price'])
            pred.append(price)
        else:
            pred.append(None)
    else:
        pred.append(None)

print(pd.Series(pred).isnull().sum())

G:\softwares\envs\4277lab\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
G:\softwares\envs\4277lab\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


3445


In [138]:
print(len(pred))

5434


In [139]:
pred = pd.DataFrame(pred)
# vi = pred[pred.isnull()].index.tolist()
# newPred = pred[vi]
# actual = train['price'][vi]
# print(len(newPred), len(actual))

In [140]:
test = test.reset_index()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5434 entries, 0 to 5433
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         5434 non-null   int64  
 1   model         5434 non-null   object 
 2   age           5434 non-null   int64  
 3   price         5434 non-null   float64
 4   dereg_value   5434 non-null   float64
 5   depreciation  5434 non-null   float64
 6   engine_cap    5434 non-null   float64
 7   omv           5434 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 339.8+ KB


In [141]:
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5434 entries, 0 to 5433
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1989 non-null   float64
dtypes: float64(1)
memory usage: 42.6 KB


In [142]:
compare = pd.concat([pred, test['price']], axis = 1).dropna()
compare.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1989 entries, 1 to 5432
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1989 non-null   float64
 1   price   1989 non-null   float64
dtypes: float64(2)
memory usage: 46.6 KB


In [143]:
pred = compare.iloc[:, 0]
actual = compare.iloc[:, 1]
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(pred, actual)
print(mse)
# 31134102722
# 511962144

670090448.9972628


In [61]:
df = pd.read_csv('test.csv')
no_dereg_but_dep = df[~df.depreciation.isnull()][df.dereg_value.isnull()]

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [64]:
diff = []
no_dereg_but_dep = no_dereg_but_dep.index.tolist()
for i in no_dereg_but_dep:
    diff.append((deps[i] - regs[i], i))
    base[i] = regs[i]

In [65]:
print(no_dereg_but_dep)

[0, 52, 85, 88, 110, 114, 124, 141, 151, 155, 214, 233, 252, 265, 269, 286, 294, 304, 305, 320, 324, 325, 331, 360, 367, 372, 411, 417, 420, 478, 511, 512, 614, 627, 649, 652, 674, 722, 763, 764, 783, 785, 801, 845, 863, 872, 898, 901, 904, 908, 921, 940, 1023, 1088, 1111, 1123, 1194, 1219, 1220, 1224, 1296, 1299, 1303, 1309, 1311, 1319, 1331, 1335, 1342, 1373, 1426, 1430, 1436, 1440, 1464, 1487, 1498, 1514, 1547, 1549, 1554, 1565, 1579, 1605, 1629, 1676, 1754, 1761, 1767, 1771, 1801, 1858, 1876, 1897, 1924, 1929, 1946, 1955, 1971, 1988, 2030, 2106, 2112, 2123, 2134, 2216, 2218, 2221, 2231, 2282, 2318, 2344, 2349, 2368, 2370, 2402, 2418, 2501, 2524, 2533, 2537, 2571, 2586, 2606, 2610, 2617, 2659, 2664, 2682, 2700, 2726, 2736, 2747, 2812, 2816, 2830, 2851, 2866, 2884, 2897, 2898, 2930, 2935, 2959, 2982, 2989, 2990, 3042, 3073, 3106, 3107, 3131, 3155, 3159, 3190, 3209, 3231, 3294, 3300, 3324, 3357, 3368, 3396, 3397, 3427, 3450, 3451, 3460, 3472, 3486, 3491, 3502, 3558, 3570, 3571, 3601, 

In [66]:
diff.sort(reverse = True)
print(diff)

[(287856.2, 325), (284706.79999999993, 4907), (217819.59999999998, 2537), (186867.40000000002, 1767), (178368.40000000002, 1876), (158144.8, 3767), (154847.19999999998, 2231), (150597.59999999998, 1897), (141627.2, 1498), (138456.8, 4677), (137296.8, 1430), (119009.20000000001, 4803), (113698.2, 1309), (106127.4, 4210), (103840.0, 872), (100948.2, 3694), (95126.79999999999, 649), (84263.0, 2898), (83626.8, 4393), (82514.0, 2700), (78627.2, 4978), (72982.4, 269), (71042.2, 3396), (68316.4, 2370), (63692.20000000001, 478), (60986.20000000001, 4119), (60214.399999999994, 4593), (58475.79999999999, 4330), (54355.0, 3621), (53946.20000000001, 908), (53912.59999999999, 3073), (53423.600000000006, 2990), (47635.79999999999, 324), (47408.399999999994, 0), (47087.600000000006, 4045), (45867.399999999994, 3601), (44558.8, 1605), (44342.8, 88), (42518.2, 3998), (41074.2, 3486), (40941.399999999994, 2106), (40427.2, 1565), (40187.0, 511), (39952.20000000001, 4974), (38041.79999999999, 4465), (3738

In [46]:
no_dep = df[df.depreciation.isnull()]
no_dep = no_dep.index.tolist()
print(no_dep)

[38, 137, 160, 194, 236, 240, 310, 371, 382, 387, 393, 430, 434, 436, 559, 565, 582, 711, 752, 753, 793, 881, 886, 888, 903, 965, 1004, 1020, 1145, 1152, 1238, 1255, 1289, 1292, 1345, 1393, 1459, 1510, 1540, 1607, 1677, 1715, 1739, 1750, 1772, 1826, 1850, 1851, 1889, 1893, 1896, 1902, 1917, 1938, 1992, 2045, 2054, 2061, 2066, 2070, 2087, 2154, 2177, 2243, 2285, 2298, 2307, 2389, 2434, 2452, 2462, 2476, 2512, 2516, 2557, 2561, 2678, 2746, 2806, 2809, 2843, 2904, 2917, 2920, 2922, 2941, 2976, 3031, 3032, 3079, 3102, 3184, 3187, 3222, 3356, 3452, 3459, 3524, 3527, 3582, 3642, 3661, 3697, 3789, 3901, 3946, 4056, 4113, 4128, 4168, 4180, 4233, 4264, 4270, 4289, 4291, 4344, 4459, 4517, 4564, 4591, 4671, 4682, 4692, 4696, 4708, 4709, 4790, 4836, 4889, 4929, 4990]


In [47]:
diff = []
for i in no_dep:
    diff.append((deps[i] - base[i], i))
    base[i] = deps[i]

In [48]:
diff.sort(reverse = True)
print(diff)

[(105589.37139999997, 2177), (76296.71429, 559), (70358.34286, 3459), (64989.34286, 1393), (61580.399999999994, 1917), (60033.257139999994, 4696), (54999.04190333331, 2285), (54843.31904333332, 4708), (52925.457140000006, 3946), (52285.62857, 2516), (50923.40476333332, 2809), (49716.71429, 137), (49534.628570000015, 2045), (47740.48571000001, 2512), (45804.57143, 4168), (44639.600000000006, 4682), (44209.342860000004, 387), (43256.142900000006, 38), (42595.17143000001, 1772), (40058.28571, 2307), (31345.942857000002, 3032), (31038.114289999998, 1152), (28661.89904333334, 1739), (27628.97143, 903), (24752.45714, 3452), (24209.45714, 1902), (23012.02857000001, 371), (22527.79999999999, 752), (20669.4, 565), (19677.227623333325, 4233), (18868.514285999998, 4671), (17855.97140000004, 753), (17117.114299999987, 1677), (16262.48570000002, 3102), (15821.085699999996, 4289), (14914.742860000006, 1292), (7211.0, 4790), (6919.457139999999, 1992), (6297.400000000023, 4564), (5800.085699999996, 43

In [36]:
print(len(no_dep))

132


In [37]:
new = pd.DataFrame(base)
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5000 non-null   float64
dtypes: float64(1)
memory usage: 39.2 KB


In [38]:
new.to_csv('combined-42-37.csv')

In [118]:
actual = [120000, 33800, 79888, 89888, 159888, 288288, 75000, 39500, 108800, 54000, 385888]
omvI = df[df.omv.isnull()].index.tolist()
omvI.remove(2633)
for i in range(len(omvI)):
    idx = omvI[i]
    hope[idx] = actual[i]

In [108]:
for i in range(len(omvI)):
    idx = omvI[i]
    if hope[idx] != actual[i]:
        print(i)

In [18]:
sr = pd.DataFrame(fill)
sr = sr.rename(columns={0: 'Predicted'})
sr.info()
# sr.to_csv('temp.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Predicted  5000 non-null   float64
dtypes: float64(1)
memory usage: 39.2 KB


In [19]:
sr.to_csv('temp.csv')

In [7]:
newDf = pd.DataFrame(fill)
newDf = newDf.rename(columns={0: 'Predicted'})
newDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Predicted  5000 non-null   float64
dtypes: float64(1)
memory usage: 39.2 KB


In [8]:
newDf.to_csv('predictions-3-8.csv')

In [62]:
hard = df[df.dereg_value.isnull()][df.depreciation.isnull()]

G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [63]:
print(len(hard))

92
